# Algorithme des k plus proches voisins

Après avoir compris le principe de l'algorithme des kPPV, nous allons  détailler les moyens de __rendre cet algorithme plus performant__.

En effet, __sans optimisation, cet algorithme peut nous donner l'illusion d'une prédiction__, sans réelle efficacité. Il est donc important de chercher à l'améliorer, en particulier en contrôlant ses résultats.

## Des caractéristiques équilibrées

Dans certains cas, les caractéristiques peuvent être déséquilibrées. Avec l'exemple choisi pour ce cours, la taille et la masse des joueurs de rugby, c'est malheureusement le cas. On le constate d'ailleurs en rappellant la relation de calcul de la distance euclidienne :

![Distance euclidienne](formule_distance.png)

Si on a souvent des écarts plus faibles entre $x_1$ et $x_2$, par rapport à la différence entre $y_1$ et $y_2$, la distance sera beaucoup plus influencée par la grandeur y. 

Dans notre cas, on a une différence de 69 kg entre le plus léger et le plus lourd des rugbymen alors que l'écart n'est que de 33 cm du plus petit au plus grand. 

La masse aura donc une importance prépondérante dans nos calculs de distance, ce qui n'est pas forcément voulu initialement.

> __Remarque :__ plus que la différence entre une valeur minimale et maximale, c'est surtout la répartition globale des données qui importe. Si cet approfondissement vous intéresse, penchez-vous sur la [notion mathématique d'écart type](https://fr.wikipedia.org/wiki/%C3%89cart_type).

Si ce n'est toujours pas assez clair, extrémisons la situation en changeant une des caractéristiques : remplaçons la taille en centimètres par une taille en micromètres (changer une unité, ne change rien à la réalité de la mesure !). 

> __Spoil :__ Ainsi, nous espérons donner plus d'importance à la taille des joueurs !

En choisissant trois joueurs aux caractéristiques extrêmes, calculons la distance entre ces joueurs, avec ou sans changement d'unités :

In [ ]:
from math import sqrt

joueur_1 = {'masse': 76, 'taille_en_cm': 173, 'taille_en_µm': 1730000}
joueur_2 = {'masse': 76, 'taille_en_cm': 206, 'taille_en_µm': 2060000}
joueur_3 = {'masse': 145, 'taille_en_cm': 173, 'taille_en_µm': 1730000}

dist_1_2_cm = sqrt((joueur_1['masse'] - joueur_2['masse'])**2 +
                (joueur_1['taille_en_cm'] - joueur_2['taille_en_cm'])**2)
dist_1_2_µm = sqrt((joueur_1['masse'] - joueur_2['masse'])**2 +
                (joueur_1['taille_en_µm'] - joueur_2['taille_en_µm'])**2)
dist_1_3_cm = sqrt((joueur_1['masse'] - joueur_3['masse'])**2 +
                (joueur_1['taille_en_cm'] - joueur_3['taille_en_cm'])**2)

print(f'Distance entre deux joueurs de masses différentes\
 : {dist_1_3_cm}')
print(f'Distance entre deux joueurs de tailles différentes\
 (distance en cm) : {dist_1_2_cm}')
print(f'Distance entre deux joueurs de tailles différentes\
 (distance en µm) : {dist_1_2_µm}')

On constate numériquement qu'avec nos données brutes, en cm, la distance entre deux joueurs de masses très différentes est deux fois plus élevée que celle entre deux rugbymen de tailles différentes. On confirme la forte influence de la masse avec nos données brutes.

Par contre, notre changement d'unités a complètement inversé la donne : la distance entre deux joueurs de tailles différente est devenue énorme.

En conséquence, pour ce pas donner plus de poids à une caractéristique qu'à une autre, il faut travailler sur les données brutes en amont. On qui qu'il faut __normaliser__ les données pour les rendre comparables.

Une façon assez simple de normaliser les données consiste à modifier chaque valeur en proportion des valeurs extrêmes de ces données : 

`valeur_normalisée = (valeur_initiale - valeur_min) / (valeur_max - valeur_min)`

> __Remarque :__ cette méthode de normalisation permet d'obtenir des valeurs uniquement comprises entre 0 et 1 inclus.

Dans le notebook 4_11 nous avons choisi une méthode s'appuyant sur une grandeur mathématique : [l'écart type](https://fr.wikipedia.org/wiki/%C3%89cart_type). Dans ce cas, les valeurs normalisées sont calculées ainsi :

`valeur_normalisée = (valeur_initiale - valeur_moyenne) / écart_type)`

## Tous les joueurs à table

Les données des joueurs sont disponibles dans un __fichier CSV__. Il faut les rendre exploitables.

__Application__

1. Exporter les données des joueurs dans une table de données nommée `joueurs`, une liste de dictionnaires. 
2. En profiter pour convertir les nombres en entiers.
3. Ajouter des valeurs de masse et de taille normalisées, par la méthode de votre choix.

In [ ]:
import csv

with open("Joueurs_rugby_6_nations.csv", mode='r', encoding='utf-8') as f:
    joueurs_reader = csv.DictReader(f, delimiter=';')
    joueurs = [{key : value for key, value in element.items()} for element in joueurs_reader]

for index in range(len(joueurs)) :
    joueurs[index]['Taille (cm)'] = int(joueurs[index]['Taille (cm)'])
    joueurs[index]['Masse (kg)'] = int(joueurs[index]['Masse (kg)'])
    
taille_min = 1000
taille_max = 0
masse_min = 1000
masse_max = 0

for index in range(len(joueurs)) :
    if joueurs[index]['Taille (cm)'] < taille_min:
        taille_min = joueurs[index]['Taille (cm)']
    if joueurs[index]['Taille (cm)'] > taille_max:
        taille_max = joueurs[index]['Taille (cm)']
    if joueurs[index]['Masse (kg)'] < masse_min:
        masse_min = joueurs[index]['Masse (kg)']
    if joueurs[index]['Masse (kg)'] > masse_max:
        masse_max = joueurs[index]['Masse (kg)']

difference_taille = taille_max - taille_min
difference_masse = masse_max - masse_min

for index in range(len(joueurs)) :
    joueurs[index]['Taille (normalisée)'] = (joueurs[index]['Taille (cm)'] - taille_min) / difference_taille
    joueurs[index]['Masse (normalisée)'] = (joueurs[index]['Masse (kg)'] - masse_min) / difference_masse

print(taille_min, taille_max, masse_min, masse_max)        
print(joueurs)

## Une structure de données équitable

Comme on l'a vu de façon extrême lorsque que k prend une valeur maximale, le résultat final dépend du nombre d'éléments présents dans chaque catégorie (chaque poste). __Si une catégorie est majoritaire, il est évident qu'elle pourra être statistiquement plus souvent voisine de notre cible__ que tout autre catégorie.

On rappelle que nos données sont basées sur des catégories inégalement réparties :

|Poste|Nombre de joueurs|
|:---:|:---:|
|Pilier|45|
|3ème lignes|44|
|2ème ligne|31|
|Centre|30|
|Talonneur|21|
|Demi de mêlée|21|
|Demi d'ouverture|21|
|Ailier|21|
|Arrière|13|

Ce __problème peut être partiellement résolu en équilibrant notre base de données__. Dans notre cas, on peut chercher de nouveaux joueurs, hors d'Europe par exemple. Au final, nous aurions 45 joueurs à chaque poste, soit au total 405 joueurs

Parfois on ne peut pas ajouter de nouveaux éléments car ils n'existent pas, il est alors __possible d'en retirer de la base existante pour effectuer notre rééquilibrage__.

Pour chaque catégorie, il faudra donc choisir...

- combien d'élément retirer.
- lesquels retirer.

Cette étape serait complexe à gérer dans notre cas :

- Si on veut avoir le même nombre de joueurs dans chaque catégorie, il faut se caler sur la plus petite catégorie (13 arrières). Mais si on réduit toutes les catégories à 13 joueurs, on passe d'une base de 247 joueurs à 117 joueurs. On perd la moitié de nos données !
- Le choix des éléments à retirer n'est pas si simple. S'il est inégalement réparti, on peut perdre de l'information importante sur la catégorie dont on va réduire l'effectif. 

Ces problèmes étant particulièrement importants lorsqu'on travaille sur des petites bases de données, pas toujours représentatives, il existe rarement une solution parfaite.

## Des éléments bien identifiés

Dans le cas d'une grande base de données, il est important de bien distinguer les éléments entre eux.

Il est donc fortement __recommandé d'associer un identifiant unique à chaque élément (`Id`)__, afin de ne pas les confondre par leur nom (en cas d'homonymie).

__Application__ : ajouter un identifiant unique `Id` à chaque joueur dans la table précédemment crées.

In [ ]:
for index in range(len(joueurs)) :
    joueurs[index]['Id'] = index + 1
    
print(joueurs)

## Un calcul de distance performant

Pour connaître les plus proches voisins, il faut __calculer la distance de chaque élément (ex : les joueurs présents dans la table de données) avec la cible à profiler (ex : Caliméro)__.

On a utilisé ce que l'on appelle une distance euclidienne. Toutefois, il existe d'autres méthodes pour calculer des distances. 

En voici une représentation graphique, sur laquelle on identifie plusieurs chemins pour mesurer la distance entre les deux points noirs.

![Distances](distances.svg)

Sur ce dessin, on remarque :
- la __distance euclidienne__ en vert.
- la __distance de Manhattan__ dans les trois autres couleurs.

Si on considère deux points $M_1$ et $M_2$ dans un plan.

- Les coordonnées de $M_1$ sont ($x_1$, $y_1$).
- Les coordonnées de $M_2$ sont ($x_2$, $y_2$).

On a vu que la __distance euclidienne__ de $M_1$ à $M_2$ est calculée ainsi :

![Distance euclidienne](formule_distance.png)

La __distance de Manhattan__ est égale à |$x_1$ - $x_2$| + |$y_1$ - $y_2$|

On utilise aussi parfois la __distance de Tchebychev__ = max(|$x_1$ - $x_2$|, |$y_1$ - $y_2$|)

![Distance de Tchebychev](tchebychev.png)

On remarque sur l'illustration ci-dessus que la distance de Tchebychev est particulièrement pertinente pour le calcul de distances d'un roi sur un échiquier.

On peut parfois aussi utiliser la [distance de Hamming](https://fr.wikipedia.org/wiki/Distance_de_Hamming).

__Application__

Calculer, sans souci  de normalisation dans un premier temps, les distances, entre ces deux joueurs (avec votre calculette ou dans la cellule de code suivante) :

- Joueur 1 : Masse 80kg, Taille 180 cm.
- Joueur 2 : Masse 70kg, Taille 175 cm.

__Application__

Coder une fonction `distance(joueur1, joueur2, methode)` qui renverra la distance entre les deux joueurs. 

Les caractéristiques des joueurs seront stockées dans un dictionnaire, sous la forme :

`{'Nom': 'Joueur 1', 'Taille (normalisée)': 180, 'Masse (normalisée)': 80}`

In [ ]:
from math import sqrt

def distance(joueur1, joueur2, methode='euclidienne'):
    return sqrt((joueur1['Taille (normalisée)'] - joueur2['Taille (normalisée)']) ** 2
                + (joueur1['Masse (normalisée)'] - joueur2['Masse (normalisée)']) ** 2)

## Une structure de données adaptée aux kPPV

Pour conserver la distance calculée, on peut tout simplement ajouter une nouvelle clé `Distance` dans l'enregistrement de chaque joueur.

> __Remarque :__ on peut également créer une nouvelle structure, une liste, qui contiendra les distances de tous les joueurs. Cette liste devra associer un identifiant unique des joueurs avec leur distance.

__Application__

On rappelle que Caliméro mesure 1m89 et pèse 102 kg.

Faite en sorte qu'une structure de données contienne toutes les distances calculées.

In [ ]:
def ajout_distances(tab, joueur_inconnu):
    for joueur in tab:
        joueur['Distance'] = distance(joueur_inconnu, joueur)
    return tab

calimero = {'Id': 1000, 'Taille (cm)': 189, 'Masse (kg)': 102}
calimero['Taille (normalisée)'] = (calimero['Taille (cm)'] - taille_min) / difference_taille
calimero['Masse (normalisée)'] = (calimero['Masse (kg)'] - masse_min) / difference_masse
 
joueurs = ajout_distances(joueurs, calimero)
print(joueurs)

## Qui sont ses k plus proches voisins ?

Une fois les distances calculées et conservées, il suffit d'en __sortir les k plus petites distances pour connaître les k plus proches voisins__.

__Deux méthodes__ peuvent s'appliquer :

- un simple tri croissant. Il suffit de conserver les k premières valeurs de la liste triée.
- un tri croissant des k premières valeurs de distance dans une nouvelle structure de données. Ensuite, on ne place une nouvelle valeur dans cette structure que si la prochaine distance examinée est plus petite que la plus grande des distances déjà stockée.

__Application__

Extraire les 3 plus proches voisins de votre précédente structure de données contenant les distances.

In [ ]:
k = 3

voisins = sorted(joueurs, key=lambda x: x['Distance'])
print(voisins[:k])

## Au final... on le met où Caliméro ?

Il suffit maintenant d'__affecter Caliméro au poste majoritaire parmi ses 3 plus proches voisins__.

__Application__

Automatiser cette recherche de catégorie majoritaire.

In [ ]:
def meilleur_poste(tab):
    postes = {}
    for voisin in tab:
        if voisin['Poste'] in postes:
            postes[voisin['Poste']] += 1
        else:
            postes[voisin['Poste']] = 1
    print(postes)
    maximum = 0
    for poste, nb in postes.items():
        if nb > maximum:
            maximum = nb
            top_poste = poste
    return top_poste

print(meilleur_poste(voisins[:k]))


## Que se passe-t-il si on change de k ?

Tester votre programme avec différentes valeurs de k.

> __Commentaires :__

Que se passe-t-il en particulier lorsque k = 7 ?

> __Commentaires :__


## Que se passe-t-il en cas d'égalité ?

En cas d'égalité d'occurrence de catégorie, il faut bien choisir. 

__Plusieurs choix__ s'offrent à nous :

- Garder la première fois que l'on rencontre cette occurrence.
- Garder la dernière fois que l'on rencontre cette occurrence.
- Laisser faire le hasard avec un random sur les ex-aequo.
- Ajouter un critère pour les départager :
  - On peut décider quel critère est prioritaire pour faire notre choix : la masse ou la taille ? (Je conseille la masse)
  - On peut pondérer les résultats inversement à leur distance.
  - On recalcule des distances sur ce critère prioritaire, uniquement pour les joueurs voisins de catégories ex-aequo.
  - On classe à nouveau ces plus proches voisins d'après cette nouvelle distance.
  - On procède à l'identique pour déterminer la catégorie majoritaire (en espérant qu'il n'y ait pas une nouvelle égalité !).
  - Et bien d'autres possibilités encore...

## La validation croisée pour déterminer le k le plus pertinent

Le meilleur moyen à notre portée pour déterminer la valeur de k la plus pertinente est d'effectuer une validation croisée.

__La validation croisée consiste à__ :

- __Extraire aléatoirement une partie des données__ initiales (ex : 1/4) et les garder pour la phase de test : on appellera cette partie les __"données-test"__.
- __Exécuter l'algorithme__ des kPPV à l'aide des données restantes (3/4 restants) __en utilisant chaque joueur conservé dans les données-test comme joueur cible__ (en remplacement de Caliméro).
- __Vérifier si l'algorithme propose le même poste__ que celui que le joueur a réellement. Bref, vérifier si l'algorithme est pertinent sur ce joueur.
- Calculer le __pourcentage de cas où l'algorithme a trouvé le réel poste__ du joueur.
- __Refaire__ de nombreuses fois l'ensemble de ce processus, __en changeant de données-test__ (un autre quart aléatoire) à chaque fois.

Ainsi, on dispose d'un __pourcentage représentatif de l'efficacité__ de notre algorithme.

On peut alors effectuer cette __validation croisée pour différentes valeurs de k__ et constater quelle valeur permet d'obtenir la meilleure efficacité.

__Applications__

1. Coder cette validation croisée pour trouver la meilleure valeur de k. 
2. Avec cette valeur de k optimale, quel pourcentage de réussite obtenez-vous pour votre algorithme prédictif ?

In [ ]:
from random import randint

# Extraction d'un tiers des données pour test de validation  

def creation_donnees_test(tab):
    joueurs_test = []
    copie_joueurs = joueurs[:]
    for _ in range(len(copie_joueurs) // 4):
        joueurs_test.append(copie_joueurs.pop(randint(0, len(copie_joueurs) - 1)))
    return joueurs_test, copie_joueurs

def distance(joueur1, joueur2, methode='euclidienne'):
    return sqrt((joueur1['Taille (normalisée)'] - joueur2['Taille (normalisée)']) ** 2
                + (joueur1['Masse (normalisée)'] - joueur2['Masse (normalisée)']) ** 2)

def ajout_distances(tab, joueur_inconnu):
    for joueur in tab:
        joueur['Distance'] = distance(joueur_inconnu, joueur)
    return tab

def meilleur_poste(tab):
    postes = {}
    for voisin in tab:
        if voisin['Poste'] in postes:
            postes[voisin['Poste']] += 1
        else:
            postes[voisin['Poste']] = 1
    maximum = 0
    for poste, nb in postes.items():
        if nb > maximum:
            maximum = nb
            top_poste = poste
    return top_poste

nb_tests = 100

for k in range(1, 31):
    bingo = 0
    for test in range(nb_tests):
        joueurs_test, joueurs_reference = creation_donnees_test(joueurs)
        for joueur_cible in joueurs_test:
            joueurs_reference = ajout_distances(joueurs_reference, joueur_cible)
            voisins = sorted(joueurs_reference, key=lambda x: x['Distance'])
            if meilleur_poste(voisins[:k]) == joueur_cible['Poste']:
                bingo += 1
    print(f"Pourcentage de réussite avec k = {k} : {round(bingo / len(joueurs_test))}")

## Complexité de l'algorithme des kPPV

La complexité de l'algorithme kPPV repose sur la __complexité du tri__ de la table. 

Par exemple, si on utilise la __fonction `sorted()`__ pour effectuer le tri de la table, la complexité de l'algorithme kPPV sera la même que celle de la fonction `sorted()`, c'est à dire __O(nLog(n))__, d'après la documentation du langage Python. 

Dans ce cas on parle de complexité quasi-linéaire.

## Que retenir ?
### À minima...

- Pour prédire la catégorie d’un nouvel élément à l'aide de l'algorithme des k plus proches voisins, il faut :
  - Un échantillon de données suffisamment large et varié.
  - Un nouvel élément dont on connaît les caractéristiques et dont on veut prédire la catégorie.
  - La valeur de k, le nombre de voisins à conserver.
- L'algorithme des kPPV se modélise ensuite ainsi :
  - Calculer les distances entre le nouvel élément et tous les éléments de l'échantillon.
  - Trouver, dans l’échantillon, les k plus proches voisins de l'élément à catégoriser.
  - Parmi ces proches voisins, trouver la catégorie majoritaire.
  
### Au mieux...

- Le choix de k est déterminant pour obtenir des résultats pertinents :
  - avec un trop petit k, l'effet loupe est trop important, ce qui laisse une trop grande part à l'aléatoire de la répartition des éléments
  - avec un trop grand k, on perd l'intérêt d'avoir un bon profil de cible et le résultat tendra vers la catégorie globalement majoritaire.

---
[![Licence CC BY NC SA](https://licensebuttons.net/l/by-nc-sa/3.0/88x31.png "licence Creative Commons CC BY-NC-SA")](http://creativecommons.org/licenses/by-nc-sa/3.0/fr/)
<p style="text-align: center;">Auteur : David Landry, Lycée Clemenceau - Nantes</p>
<p style="text-align: center;">D'après des documents partagés par...</p>
<p style="text-align: center;"><a  href=http://www.monlyceenumerique.fr/index_nsi.html#premiere>JC. Gérard, T. Lourdet, J. Monteillet, P. Thérèse, sur le site monlyceenumerique.fr</a></p>
<p style="text-align: center;"><a  href=https://www.infoforall.fr/act/algo/k-plus-proches-voisins/>Infoforall</a></p>
<p style="text-align: center;"><a  href=https://www.sixnationsrugby.com/>Les données des rugbymen ont été trouvées sur le site officiel sixnationsrugby.com</a></p>
<p style="text-align: center;"><a  href=http://www.bamboo.fr/lesrugbymen/>L'image des positions des joueurs est tirée de la BD Rugbymen, de Beka et Poupard</a></p>